In [1]:
import os
import sys
import re
from pathlib import Path
import time
import warnings
import logging
import pandas as pd
import numpy as np
import pickle
import json
from matplotlib import pyplot as plt

sys.path.append('../python')

import api
import data_tools as dt
import text_tools as tt

rng = np.random.default_rng(12898)


/Users/ekung/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ekung/Library/Python/3.9/lib/python/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
df = dt.get_minutes(verbose=False)

In [3]:
case_list = []
for idx, row in df.iterrows():
    title = row['title']
    cases = row['related_cases'].split(',')
    for caseno in [title]+cases:
        if tt.is_casenum(caseno.strip()):
            case_list.append(caseno.strip())
case_list = sorted(list(set(case_list)))

In [4]:
out_df = []
count=0
for case in case_list:
    j = api.get_case_info(case, verbose=True, wait_time=2)
    if j is not None:
        j['casenum'] = case
        out_df.append(j)
        count+=1

out_df = pd.DataFrame.from_dict(out_df)
out_df.to_pickle("../../intermediate_data/cpc/case-data.pkl")    

In [7]:
# Ensure the case numbers line up
out_df['bad'] = False
for idx, row in out_df.iterrows():
    caseNbr = row['caseNbr']
    casenum = row['casenum']
    if api.canonicalize(caseNbr)!=api.canonicalize(casenum):
        out_df.loc[idx, 'bad'] = True
        
assert out_df['bad'].sum()==0

In [19]:
# Ensure the agenda items can be found
df2 = df.merge(
    out_df[['caseNbr','currentCaseStatus','casenum']],
    left_on='title',
    right_on='casenum',
    how='left'
)

idx = df2['caseNbr'].isna()
df2.loc[idx, ['date', 'item_no', 'title', 'related_cases']]

,date,item_no,title,related_cases
83,2019-01-10,6,ADM-2018-5752-DB-SIP,ADM-2018-5752-DB-SIP
415,2021-12-09,10,ADM-2021-3739-DB-HCA-1A,ADM-2021-3739-DB-HCA-1A
478,2022-07-14,8,ADM-2021-10304-DB-HCA-1A,ADM-2021-10304-DB-HCA-1A
553,2023-04-27,6,ADM-2022-6793-DB-HCA-1A,ADM-2022-6793-DB-HCA-1A
643,2024-02-22,8,ADM-2023-5502-DB-HCA-1A,ADM-2023-5502-DB-HCA-1A


In [24]:
df2['level'] = ''
for idx, row in df2.iterrows():
    level = row['title'].split('-')[0]
    df2.loc[idx,'level'] = level

In [25]:
idx = (df2['currentCaseStatus']=='DENIED') & (df2['project_result']=='APPROVED')
df2.loc[idx, 'level'].value_counts()

level
DIR    29
VTT    18
CPC     1
ZA      1
AA      1
Name: count, dtype: int64

In [ ]:
# In most of the disagreements between the PDIS data and the AI data, the AI is actually correct
# The project was approved, but for some reason the PDIS says the case is DENIED.
# I think the PDIS may be referencing the appeal(?)
print(df2.loc[idx].sample(1,random_state=329).iloc[0]['prompt'])